<a href="https://colab.research.google.com/github/Goodbam/Changing-Naver-rising-searches/blob/master/crawling(12_5_2AM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
!pip install lxml
!pip install parse

In [0]:
import requests                 # url request
from lxml.html import parse     # parsing the data as the html form
from io import StringIO         # String I/O module

THRESHOLD = 16 #limit to check whether they are relative or not
NOISE = 9

In [75]:
url = "https://www.naver.com"

# Get html source
text = requests.get(url).text

# parsing the data as the html form
text_source = StringIO(text)    # read the data as string
parsed = parse(text_source)     # source ->parsing the data as the html form 

#find root node
doc = parsed.getroot()

# doc.findall(".//TagName") => find the tag lists
uls = doc.findall('.//ul')

#Get the Rising searches
ul = uls[10]   
spans = ul.findall('.//span')

data_list= []  # to save all the keywords

cnt = 1
for data in spans:
  if cnt % 2 == 0:  
    print(int((cnt+1)/2),'->',data.text_content().strip())  # show the existing rising searches
  cnt +=1  
  data_list.append(data.text_content().strip()) #save each keyword
    


1 -> 핑거루트
2 -> 백석역
3 -> 이선정
4 -> 홍현희 제약회사
5 -> 서인영 사건
6 -> 양수경 남편
7 -> 나이트 워치맨
8 -> 프리즌
9 -> 정가은
10 -> 황지만
11 -> 이일재
12 -> lj
13 -> 한정수
14 -> 나리미야 히로키
15 -> 이설
16 -> 미미쿠키
17 -> 제이쓴
18 -> 스니치
19 -> 2019 수능 등급컷
20 -> 신아영


In [76]:
path='https://search.naver.com/search.naver?sm=top_hty&fbm=1&ie=utf8&query=' #This is the url for surfing

matrix = [] #to count the score of relationship
label = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] #disjoint set

cnt1 = 0

for i in range(0, 20):
  word = data_list[1 + cnt1]
  search = path + word #url searching the keyword
  text = requests.get(search).text # no worry about encoding url
  
  row = []
  cnt2 = 0
  for j in range(0, 20):
    if cnt1 != cnt2: #no need to check when they are equivalent
      target = data_list[1 + cnt2];
      row.append(text.count(target) + 7*word.count(target) - NOISE) 
 
      if row[int(cnt2/2)] >= THRESHOLD: #IF they are relative,
        print(word, ' and ', target,)
        #the representive number should be minimized
        
        #여기 정규화를 해서 row의합으로 각 row의 값을 나누던지 해보자
        #그러려니, 전체적으로 점수 자체가 높은 것은 감안해서 평균을 다시 루트씌우고 곱하든지 고려해보자
        #A와 B 관련 , B C 관련 이면 A C는 관련이 있는건가? 이것도 감안
        
        if label[int(cnt1/2)] < label[int(cnt2/2)]:
          label[int(cnt2/2)] = label[int(cnt1/2)]  
        
        else:
          label[int(cnt1/2)] = label[int(cnt2/2)]
    
    else :
      row.append(0)
    
    cnt2 += 2
  
  matrix.append(row)  
  cnt1 += 2
  

이선정  and  정가은
이선정  and  황지만
이선정  and  lj
홍현희 제약회사  and  제이쓴
양수경 남편  and  한정수
프리즌  and  lj
황지만  and  이선정
lj  and  이선정
lj  and  황지만


In [77]:
print('\n',label,'\n')

print(matrix)

size = [0]
size = size*20

for i in range(0,20):
  size[i] = sum(matrix[i])

print(size)
score =[0]
score = score*20

for i in range(0,20):
  score[label[i] - 1] += 1/(i+1); #label is 1~20 => needed to subtract 1


 [1, 2, 3, 4, 5, 6, 7, 3, 3, 3, 11, 3, 6, 14, 15, 16, 4, 18, 19, 20] 

[[0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 13, 3, 0, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 10, 2, 0, 0, 0, 0, 0, 0, 0], [2, 2, 0, 0, 0, 0, 0, 0, 20, 24, 2, 20, 2, 0, 0, 0, 0, 0, 0, 0], [2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 12, 2, 0, 0, 0, 21, 0, 0, 0], [2, 2, 6, 0, 0, 0, 0, 0, 0, 0, 2, 14, 2, 0, 0, 0, 0, 0, 0, 0], [2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 8, 30, 0, 0, 0, 0, 0, 0, 0], [2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 7, 2, 0, 0, 0, 0, 0, 0, 0], [2, 2, 2, 0, 0, 0, 0, 0, 0, 2, 2, 19, 2, 0, 0, 0, 0, 0, 0, 0], [2, 2, 8, 0, 0, 0, 0, 0, 0, 0, 2, 7, 2, 0, 0, 0, 0, 0, 0, 0], [2, 2, 112, 0, 0, 0, 0, 0, 2, 0, 2, 13, 2, 0, 0, 0, 0, 0, 0, 0], [2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 2, 0, 1, 0, 0, 0, 0, 0], [2, 2, 63, 0, 0, 0, 0, 0, 5, 27, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0], [2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 7, 0, 0, 0, 0, 0, 0, 0, 0], [2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 7, 2, 0, 0, 0, 0, 0, 0, 0], [2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 7, 2, 0, 0

In [78]:
#RESULT
print('\n☆급상승 검색어 순위☆\n')
cnt = 1;

#sorting according to SCORE
for i in range(0,20):
  max = 0
  for j in range(1,20):
    if score[j] > score[max]:
      max = j;
  
  relative_word = ''
  
  for j in range(0,20):
    if label[j] == max + 1:
      relative_word += data_list[1 + 2*j] + ' | ' 
  
  score[max] = 0;
  
  print(cnt, '. ', relative_word[0:-3])
  cnt += 1
  if(score.count(0) == 20):
    break  


☆급상승 검색어 순위☆

1 .  핑거루트
2 .  이선정 | 프리즌 | 정가은 | 황지만 | lj
3 .  백석역
4 .  홍현희 제약회사 | 제이쓴
5 .  양수경 남편 | 한정수
6 .  서인영 사건
7 .  나이트 워치맨
8 .  이일재
9 .  나리미야 히로키
10 .  이설
11 .  미미쿠키
12 .  스니치
13 .  2019 수능 등급컷
14 .  신아영
